In [1]:
import sys
import numpy as np
np.set_printoptions(suppress=True, precision=4)
import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.4f}'.format
import seaborn as sns
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import log_loss, make_scorer, accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.utils import class_weight
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import IsolationForest
from sklearn.metrics import log_loss, make_scorer
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.metrics import BinaryCrossentropy
from methods import *
import torch
import pickle, os
import skopt
from skopt import BayesSearchCV, gp_minimize
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
import shutil
import time
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
print('done')

done


In [2]:
%config IPCompleter.greedy=True

In [3]:
%load_ext autoreload
%autoreload 2
print('done')

done


# Train Neural Network

In [4]:
def create_neural_network(input_size):
    model = Sequential()
#     model.add(Dense(7, input_dim=input_size, activation='softmax')) # logistic regression: only one dense layer with 7 units, input_size and softmax    

    model.add(Dense(10, input_dim=input_size, activation='relu')) # logistic regression: only one dense layer with 7 units, input_size and softmax
#     model.add(Dense(30, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def cross_validation(data_x, data_y):
    n_splits = 4
    n_repeats = 3
    pipeline = Pipeline([('scale', MinMaxScaler()), ('mlp', KerasClassifier(build_fn=create_neural_network, epochs=100, batch_size=32, verbose=0))])
    for _ in range(3):
        pipeline = Pipeline([('mlp', KerasClassifier(build_fn=create_neural_network, epochs=50, batch_size=16, verbose=0))])
        outer_kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=666)
        for idx_train_val, idx_test in outer_kfold.split(data_x, data_y):
            subset_data_x = data_x[idx_train_val, :]
            subset_data_y = data_y[idx_train_val]
            inner_kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=666)
# #     weights = np.ones((7,))
# #     weights = class_weight.compute_class_weight('balanced', np.unique(data_y), data_y)#/ n_splits
#     pipeline = Pipeline([('mlp', KerasClassifier(build_fn=create_neural_network, epochs=50, batch_size=8, verbose=1))])
#     kfold = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=666)
#     results = cross_val_score(pipeline, data_x, data_y, cv=kfold, scoring='accuracy', n_jobs=-1)#, fit_params={'sample_weight': weights}) 
#     print(f'CV: acc mean={results.mean():.3f}, acc std={results.std():.3f}')
# #     results = results[~np.isnan(results)]
# #     results = results[~np.isinf(results)]
# #     print(f'CV: CE mean={-results.mean():.3f} CE std={results.std():.3f}')
    
def train_nn(data_x, data_y, test_x=None, test_y=None):
    data_y_onehot = OneHotEncoder().fit_transform(data_y.reshape(-1,1)).toarray()
    test_y_onehot = OneHotEncoder().fit_transform(test_y.reshape(-1,1)).toarray()
    
#     scaler = StandardScaler()
#     scaler.fit(data_x[:,:12])
#     data_x = scaler.transform(data_x[:,:12])
#     test_x = scaler.transform(test_x[:,:12])

    # convert labels from {0, ..., 6} to {0, 1}. Backdoored classes in {1, ..., 6} are mapped to class 1 (simply backdoored)
    ## for training data
    data_y_binarized = np.copy(data_y)
    data_y_binarized[data_y_binarized > 0] = 1
    ## for holdout data
    test_y_binarized = np.copy(test_y)
    test_y_binarized[test_y_binarized > 0] = 1
    
    model = create_neural_network(input_size=data_x.shape[1])
    
    weights = None # dict(enumerate(class_weight.compute_class_weight('balanced', np.unique(data_y), data_y)))
    history = model.fit(data_x, data_y_onehot, epochs=100, batch_size=8, verbose=0, class_weight=weights)
    print('Class weights:', weights)
    
    # get one-hot prediction on training data
    pred_train = model.predict(data_x)
    
    # get predicted classes (index of maximum probability)
    pred_train_labels = np.argmax(pred_train, axis=1)
    
    # convert labels from {0, ..., 6} to {0, 1}. Backdoored classes in {1, ..., 6} are mapped to class 1 (simply backdoored)
    pred_train_binarized = np.copy(pred_train_labels)
    pred_train_binarized[pred_train_binarized > 0] = 1

    # keep clean probability as it is and compute the backdoored probability as 1-clean probability just like we had binary classification
    pred_train_probas = np.zeros((data_x.shape[0], 2))
    pred_train_probas[:, 0] = pred_train[:, 0]
    pred_train_probas[:, 1] = 1 - pred_train[:, 0]
    
    loss, accuracy = model.evaluate(data_x, data_y_onehot, verbose=0)
    train_ce = log_loss(y_true=data_y_binarized, y_pred=pred_train_probas)
    print(f'\nTrain Loss:      {loss:.3f}')
    print(f'Train Accuracy:  {accuracy_score(y_true=data_y, y_pred=pred_train_labels):.3f}')
    print(f'Train Precision: {precision_score(y_true=data_y_binarized, y_pred=pred_train_binarized):.3f}')
    print(f'Train Recall:    {recall_score(y_true=data_y_binarized, y_pred=pred_train_binarized):.3f}')
    print(f'Train Log-Loss:  {train_ce:.3f}')
    print(f'Train Confusion Matrix (all):\n{confusion_matrix(y_true=data_y, y_pred=pred_train_labels)}')
    print(f'Train Confusion Matrix (binary):\n{confusion_matrix(y_true=data_y_binarized, y_pred=pred_train_binarized)}')
    
    if test_x is not None and test_y is not None:
        pred_holdout = model.predict(test_x)
        
        # get one-hot prediction on training data
        pred_holdout_labels = np.argmax(pred_holdout, axis=1)
        
        # convert labels from {0, ..., 6} to {0, 1}. Backdoored classes in {1, ..., 6} are mapped to class 1 (simply backdoored)
        pred_holdout_binarized = np.copy(pred_holdout_labels)
        pred_holdout_binarized[pred_holdout_binarized > 0] = 1
        
        # keep clean probability as it is and compute the backdoored probability as 1-clean probability just like we had binary classification
        pred_holdout_probas = np.zeros((test_x.shape[0], 2))
        pred_holdout_probas[:, 0] = pred_holdout[:, 0]
        pred_holdout_probas[:, 1] = 1 - pred_holdout[:, 0]
        
        loss, accuracy = model.evaluate(test_x, test_y_onehot, verbose=0)
        holdout_ce = log_loss(y_true=test_y_binarized, y_pred=pred_holdout_probas)
        print(f'\nTest Loss:      {loss:.3f}')
        print(f'Test Accuracy:  {accuracy_score(y_true=test_y, y_pred=pred_holdout_labels):.3f}')
        print(f'Test Precision: {precision_score(y_true=test_y_binarized, y_pred=pred_holdout_binarized):.3f}')
        print(f'Test Recall:    {recall_score(y_true=test_y_binarized, y_pred=pred_holdout_binarized):.3f}')
        print(f'Test Log-Loss:  {holdout_ce:.3f} <========')
        print(f'Test Confusion Matrix (all):\n{confusion_matrix(y_true=test_y, y_pred=pred_holdout_labels)}')
        print(f'Test Confusion Matrix (binary):\n{confusion_matrix(y_true=test_y_binarized, y_pred=pred_holdout_binarized)}')
        
#     folder = r'D:\Cloud\MEGA\TrojAI\TrojAI-UMD\metamodels\metamodel_10_round3_NN-60-30_min-max-scaled_RAW_square30_RANDOM_filters_all-classes'
#     folder = r'D:\Cloud\MEGA\TrojAI\TrojAI-UMD\metamodels\metamodel_11_round3_NN-60-30_min-max-scaled_RAW_square30_GRAY'

#     the_time = datetime.now().strftime('%Y-%m-%d_%Hh%Mm%Ss')
#     folder = fr'neural_networks_separate_backdoors_round_4/{the_time}_train-CE={train_ce:.3f}_holdout-CE={holdout_ce:.3f}_input={data_x.shape[1]}'
#     keras_save(model, folder)
    
#     save_obj(scaler, os.path.join(folder, 'scaler.pickle'))
    return model

path_train = r'confusion-reports\ics_fc\round4-train-dataset\round4-train-dataset_fc_synthetic_polygon-all-gray_filters_0-1007.csv'

use_arch = True
X_0, y_0 = read_features(path_train, trigger_type_aux_str=None, arch=None, data='diffs', label_type='backdoor_code_0', append_arch=use_arch, arch_one_hot=use_arch)
X_1, y_1 = read_features(path_train, trigger_type_aux_str=None, arch=None, data='diffs', label_type='backdoor_code_1', append_arch=use_arch, arch_one_hot=use_arch)

percent_training = 0.7
n_samples = X_0.shape[0]
np.random.seed(int(time.time()))
# np.random.seed(666)
indexes_train = np.random.choice(n_samples, int(n_samples * percent_training), replace=False).tolist()
indexes_test = [i for i in range(n_samples) if i not in indexes_train]
print('indexes_test:', indexes_test)

X_0_train, y_0_train, X_0_test, y_0_test = X_0[indexes_train, :], y_0[indexes_train], X_0[indexes_test, :], y_0[indexes_test]
X_1_train, y_1_train, X_1_test, y_1_test = X_1[indexes_train, :], y_1[indexes_train], X_1[indexes_test, :], y_1[indexes_test]

print(X_0_train.shape, y_0_train.shape, X_0_test.shape, y_0_test.shape)
# print(X_0_train[0, :], y_0_train[0])
# print(X_0_test[0, :], y_0_test[0])
# print()
print(X_1_train.shape, y_1_train.shape, X_1_test.shape, y_1_test.shape)
# print(X_1_train[0, :], y_1_train[0])
# print(X_1_test[0, :], y_1_test[0])

# cross_validation(X_train, y_train)
print('#################### BACKDOOR 0 ####################')
model_0 = train_nn(X_0_train, y_0_train, X_0_test, y_0_test)
print('#################### BACKDOOR 1 ####################')
model_1 = train_nn(X_1_train, y_1_train, X_1_test, y_1_test)
print(f'Using arch: {use_arch}')

pred_0 = model_0.predict(X_0_test)
pred_1 = model_1.predict(X_1_test)

df = pd.DataFrame(columns=['true_binary_label', 'true_backd_0', 'true_backd_1', 'pred_backd_0', 'pred_backd_1', 'backdoor_proba', 'proba_0', 'proba_1'])
ce_loss = 0.0
for i in range(pred_0.shape[0]):
    t0, t1 = y_0_test[i], y_1_test[i]
    p0, p1 = pred_0[i, :], pred_1[i, :]
    label0, label1 = np.argmax(p0), np.argmax(p1)
    if label0 + label1 == 0: # both models predicted 'clean'
        backdoor_proba = 1.0 - (p0[0] + p1[0]) / 2.0 # mean of the two clean predictions (1 - value) = backdoor probability
#     elif label0 == 0:
#         backdoor_proba = p1[1:].sum()
#     elif label1 == 0:
#         backdoor_proba = p0[1:].sum()
    else:
        backdoor_proba = (p0[1:].sum() + p1[1:].sum()) / 2.0
    t = 0 if t0 + t1 == 0 else 1
    ce_loss += -(t * np.log(backdoor_proba) + (1-t) * np.log(1.0 - backdoor_proba))
    df.loc[i] = [t, t0, t1, label0, label1, backdoor_proba, list(map(lambda x: round(x,2), p0.tolist())), list(map(lambda x: round(x,2), p1.tolist()))]
ce_loss = ce_loss / pred_0.shape[0]
print(f'CE loss = {ce_loss}')
the_time = datetime.now().strftime('%Y-%m-%d_%Hh%Mm%Ss')
folder = fr'neural_networks_separate_backdoors_round_4/{the_time}_test-CE={ce_loss:.3f}_input={X_0.shape[1]}_train-count={len(indexes_train)}_test-count={len(indexes_test)}'
keras_save(model_0, folder, name='model_0')
keras_save(model_1, folder, name='model_1')
print('done')
# df[['true_binary_label', 'true_backd_0', 'true_backd_1', 'pred_backd_0', 'pred_backd_1']] = df[['true_binary_label', 'true_backd_0', 'true_backd_1', 'pred_backd_0', 'pred_backd_1']].astype('int')
# df

Using diffs
Using diffs
indexes_test: [0, 1, 2, 8, 9, 11, 15, 19, 20, 23, 25, 28, 32, 34, 39, 42, 44, 60, 62, 65, 71, 74, 76, 78, 81, 85, 88, 90, 95, 96, 99, 102, 103, 106, 108, 109, 115, 124, 131, 134, 135, 136, 146, 148, 154, 155, 161, 162, 164, 165, 166, 170, 171, 177, 179, 182, 183, 184, 188, 198, 202, 204, 206, 218, 219, 222, 226, 232, 234, 236, 237, 240, 241, 243, 244, 245, 247, 254, 255, 259, 264, 266, 269, 270, 272, 274, 282, 286, 287, 288, 289, 290, 298, 301, 307, 310, 315, 317, 324, 325, 332, 337, 340, 346, 347, 351, 354, 355, 363, 366, 370, 372, 379, 380, 382, 385, 386, 390, 393, 395, 396, 399, 402, 403, 404, 408, 416, 419, 421, 422, 424, 440, 453, 454, 455, 463, 465, 467, 471, 472, 477, 479, 480, 484, 487, 488, 489, 498, 499, 503, 504, 519, 521, 525, 536, 539, 541, 548, 553, 555, 557, 559, 567, 572, 578, 579, 582, 587, 589, 591, 594, 597, 603, 604, 609, 611, 612, 614, 617, 619, 621, 622, 623, 625, 627, 628, 629, 633, 644, 645, 648, 650, 654, 655, 661, 662, 665, 677, 681, 68

# Explore dataset

In [ ]:
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-random_filters.csv'
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-random_filters.csv'
path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-30-gray_filters_h_kl.csv'
df = pd.read_csv(path_csv)
df.describe()

In [ ]:
path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-30-random_filters_h_kl.csv'
df = pd.read_csv(path_csv)
labels = df['model_label']
for x in ['model_name', 'model_architecture', 'architecture_code', 'trigger_color', 'num_classes', 'trigger_type_aux']:
    del df[x]
for k in ['clean', 'square30', 'gotham', 'kelvin', 'lomo', 'nashville', 'toaster']:
    df[f'ch_{k}'] = df[f'h_{k}'] + df[f'kl_{k}']
#     del df[f'h_{k}'], df[f'kl_{k}']
plt.figure(figsize=(40,20)).patch.set_color('white')
sns.heatmap(df.corr(), annot=True, cmap='YlGnBu')
# df.head()

In [ ]:
# path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-30-gray_filters_h_kl.csv'
path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-30-random_filters_h_kl.csv'
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-random_filters.csv'
X, y = read_features(path_csv, trigger_type_aux_str=None, label_type='binary', arch=None, data='hkl', append_arch=False, arch_one_hot=False)
# X, y = read_features(path_csv, trigger_type_aux_str='polygon', arch=None, data='diffs', append_arch=False, arch_one_hot=False)
print(X.shape)
# X = StandardScaler().fit_transform(X)
arr = ['h_clean', 'kl_clean', 'h_square30', 'kl_square30', 'h_gotham', 'kl_gotham', 'h_kelvin', 'kl_kelvin', 'h_lomo', 'kl_lomo', 'h_nashville', 'kl_nashville', 'h_toaster', 'kl_toaster']
# arr = ['square30_r_mean_diff', 'square30_r_std_diff', 'gotham_mean_diff', 'gotham_std_diff', 'kelvin_mean_diff', 'kelvin_std_diff', 'lomo_mean_diff', 'lomo_std_diff', 'nashville_mean_diff', 'nashville_std_diff', 'toaster_mean_diff', 'toaster_std_diff']
print(list(zip(arr, X[0,:].tolist())))

i, j = arr.index('kl_square30'), arr.index('kl_toaster')
# i, j = arr.index('square30_r_mean_diff'), arr.index('square30_r_std_diff')

f1, f2 = abs(X[:, i]), abs(X[:, j])
f1 = np.log(f1)
f2 = np.log(f2)
plt.figure(figsize=(20,15)).patch.set_color('white')
plt.scatter(f1, f2, c=[['blue', 'red'][vy] for vy in y], s=15.0)
# plt.legend()
plt.grid()
# Good results for H+KL:
# indexes=3,4, log(|f1|), log(|f2|), h+kl (kl_square30, h_gotham)
# indexes=3,5 (kl_square30, kl_gotham)
# indexes=3,5 (kl_square30, kl_toaster)

# with standardization:
# 3, 4, log f1, log f2

## Correlation Plot

In [ ]:
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-filters_all-classes_gray.csv'
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-random_filters.csv'
# path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-30-gray_filters_h_kl.csv'
path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-30-random_filters_h_kl.csv'
df = pd.read_csv(path_csv)
df = df[df['model_architecture'].str.contains('vgg')]
df['model_label'] = 1 - df['model_label'].astype('category').cat.codes
del df['num_classes']
indexes = []
arch = None
if arch is not None:
    for i in range(len(df)):
        if df['model_architecture'].iloc[i].startswith(arch):
            indexes.append(i)
    df = df.iloc[indexes]
for c in df.columns:
    if c.endswith('mean') or c.endswith('std'):# or c.startswith('h'):
#     if c.endswith('diff'):
        del df[c]
print(df.shape)
print(df.iloc[0])

for k in ['clean', 'square30', 'gotham', 'kelvin', 'lomo', 'nashville', 'toaster']:
    df[f'kl_{k}'] = np.log(df[f'kl_{k}'])
    df[f'h_{k}'] = np.log(df[f'h_{k}'])

plt.figure(figsize=(40,20)).patch.set_color('white')
sns.heatmap(df.corr(), annot=True, cmap='YlGnBu')

# plt.figure(figsize=(30,12)).patch.set_color('white')
# sns.pairplot(df, hue='model_label')

## Box-plots

In [ ]:
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-filters_all-classes_gray.csv'
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'
path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-random_filters.csv'
# path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square30-gray_filters_h_kl.csv'
df = pd.read_csv(path_csv)
df['model_label'] = 1 - df['model_label'].astype('category').cat.codes
del df['num_classes'], df['model_name']
indexes = []
arch = None
if arch is not None:
    for i in range(len(df)):
        if df['model_architecture'].iloc[i].startswith(arch):
            indexes.append(i)
    df = df.iloc[indexes]
for c in df.columns:
    if c.endswith('mean') or c.endswith('std'):
#     if c.endswith('diff'):
        del df[c]
del df['model_architecture'], df['architecture_code'], df['trigger_type_aux'], df['trigger_color']#, df['model_label']
print(df.columns.tolist())
for c in df.columns:
#     if c.startswith('h'):
    if c.endswith('mean_diff'):
        plt.figure().patch.set_color('white')
        sns.boxplot(x="model_label", y=c, hue="model_label", data=df, palette="Set3")
print('done')

In [ ]:
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-filters_all-classes_gray.csv'
path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'
df = pd.read_csv(path_csv)
df['model_label'] = 1 - df['model_label'].astype('category').cat.codes
indexes = []
arch = None
if arch is not None:
    for i in range(len(df)):
        if df['model_architecture'].iloc[i].startswith(arch):
            indexes.append(i)
    df = df.iloc[indexes]
for c in df.columns:
    if c.endswith('mean') or c.endswith('std'):
#     if c.endswith('diff'):
        del df[c]
del df['model_architecture'], df['trigger_type_aux'], df['trigger_color'], df['num_classes'], df['model_name']
# print(df.head())

data = df.values
X, y = data[:, 1:], data[:, 0]
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X)
mask = np.where(yhat == 1)[0]
print(mask.shape)
del df['model_label']
plt.figure(figsize=(30, 12)).patch.set_color('white')
df.iloc[mask].boxplot()

# Stratified K-fold validation for a full training dataset (square-size and 5 filters)

In [ ]:
n_splits = 10
n_repeats = 5

for size in [30]: #[10, 15, 20, 25, 30, 35, 40, 45, 50]:
#     path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-{size}_filters.csv'
#     path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-{size}-random_filters.csv'
#     path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-{size}-gray_filters.csv'
#     path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-{size}-random_filters.csv'
#     path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-{size}-gray_filters_h_kl.csv'
    path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-{size}-random_filters_h_kl.csv'
    for A in [None]: #['vgg', 'wideresnet', 'resnet', 'densenet', 'mobilenet', 'shufflenet', 'squeezenet', 'inception', 'googlenet']:
        X, y = read_features(path_csv, trigger_type_aux_str=None, arch=None, data='hkl', append_arch=False, arch_one_hot=False) #['vgg', 'resnet' (includes wideresnet), 'densenet', 'mobilenet', 'shufflenet', 'squeezenet', 'inception', 'googlenet', None]:
    #     X, y = X[2:,[0, 2, 4, 6]], y[2:]
    #     X, y = X[2:,:], y[2:]
        scores_roc, scores_xent = [], []
        print(X.shape, y.shape)
        print(X[0,:].tolist())
        try:
            for _ in range(n_repeats):
                kfold =  StratifiedKFold(n_splits=n_splits, shuffle=True)
                for train_index, test_index in kfold.split(X, y):
                    X_train, y_train = X[train_index, :], y[train_index]
                    X_test, y_test = X[test_index, :], y[test_index]

    #                 model = RandomForestClassifier(n_estimators=100)
                    model = RandomForestClassifier(n_estimators=500)
    #                 model = GradientBoostingClassifier(n_estimators=100, loss='deviance')
    #                 model = GradientBoostingClassifier(n_estimators=100, loss='exponential')
    #                 model = svm.SVC(C=1, probability=True, kernel='rbf')

#                     model = LogisticRegression(C=0.5)
#                     scaler = StandardScaler()
#                     scaler.fit(X_train)
#                     X_train = scaler.transform(np.copy(X_train))
#                     X_test = scaler.transform(np.copy(X_test))

                    roc, xent = evaluate_classifier(model, X_train, y_train, X_test, y_test)
                    scores_roc.append(roc)
                    scores_xent.append(xent)
        except ValueError as e:
            print('ERROR')
            print(e)
        print(f'size={size}')
        print(f'CE mean={np.mean(scores_xent):.3f}, CE std={np.std(scores_xent):.3f}')
        print('-------------------------------------------------------------------------')
print('done')

# HPO for training data

In [ ]:
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'
path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-random_filters.csv'
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30_filters.csv'
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-random_filters.csv'
X, y = read_features(path_csv, trigger_type_aux_str=None, arch=None, data='diffs', append_arch=True)
print(X.shape)
params = {
    "C": [0.01 + i * 0.01 for i in range(1000)]
}
grid_cv = GridSearchCV(estimator=LogisticRegression(),
                       param_grid=params,
                       cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1),
                       scoring=make_scorer(log_loss, greater_is_better=False, needs_proba=True),
                       n_jobs=-1)
grid_cv.fit(X, y)

# print('Best params:', grid_cv.best_params_)
# print('Best CE:', grid_cv.best_score_)
pd.DataFrame.from_dict({
    'C': [d['C'] for d in grid_cv.cv_results_['params']],
    'CE': -grid_cv.cv_results_['mean_test_score']
}).sort_values(['CE'], ascending=True, axis=0)

# Train meta-model using square data and filters

In [ ]:
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'
path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-random_filters.csv'
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-random_filters.csv'
# path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-30-random_filters_h_kl.csv'
for arch in ['vgg', 'resnet', 'densenet', 'mobilenet', 'shufflenet', 'squeezenet', 'inception', 'googlenet']:
    X_train, y_train = read_features(path_csv, trigger_type_aux_str=None, arch=arch, data='diffs', append_arch=False, arch_one_hot=False)
#     X_train = X_train[2:, :]
#     y_train = y_train[2:]
    print(arch, X_train.shape)
#     scaler=None
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)

#     model = RandomForestClassifier(n_estimators=500)
    model = LogisticRegression()
    model.fit(X_train, y_train)

    name = '../metamodels/metamodel_16_fc_round3_data=diffs_square=30-random_scaler=std_clf=LR-1_arch-features=no_arch-wise-models=yes'
#     if os.path.isdir(name):
#         shutil.rmtree(name)
    if not os.path.isdir(name):
        os.mkdir(name)
    save_obj(model, f'{name}/model-{arch}.pickle')
    save_obj(scaler, f'{name}/scaler-{arch}.pickle')
    print('done')

In [ ]:
path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'

X, y = read_features(path_csv, trigger_type_aux_str=None, arch=None, data='diffs')
X_train, y_train = X[:500, :], y[:500]
X_test, y_test = X[500:, :], y[500:]

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
model = LogisticRegression(C=1)
model.fit(X_train, y_train)
ce = 0.0
for i in range(X_test.shape[0]):
    prob = model.predict_proba(X_test[i,:].reshape(1, -1))
    p0, p1 = prob[0]
    t = y[i]
    local_ce = -(t * np.log2(p1) + (1-t) * np.log2(1-p0))
#     print(i, local_ce)
    ce += local_ce
print(ce / X_test.shape[0])

# Local Testing: train on training data and test on holdout data

In [ ]:
path_train_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray.csv'
path_holdout_csv = r'confusion-reports\ics_svm\round2-holdout-dataset\round2-holdout-dataset_square-25-filters_all-classes_gray.csv'
# path_train_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'
# path_holdout_csv = r'confusion-reports\ics_svm\round2-holdout-dataset\round2-holdout-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'
trigger_type_aux_str = None

print('Local Testing')
if 'confusion-matrix' in path_train_csv and 'confusion-matrix' in path_holdout_csv:
    print('Approach: confusion matrix and original CNN')
    X_train, y_train = read_features_confusion_matrix(path_train_csv, trigger_type_aux_str)
    X_holdout, y_holdout = read_features_confusion_matrix(path_holdout_csv, trigger_type_aux_str)
else:
    print('Approach: confusion distribution and SDNs')
    X_train, y_train = read_features(path_train_csv, trigger_type_aux_str)
    X_holdout, y_holdout = read_features(path_holdout_csv, trigger_type_aux_str)

print('train shape:', X_train.shape, y_train.shape)
print('holdout shape:', X_holdout.shape, y_holdout.shape)

roc, xent = evaluate_classifier(X_train, y_train, X_holdout, y_holdout)
print(f'ROC AUC = {roc}')
print(f'Cross-Entropy = {xent}')